# Machine Learning Interpretability




**Ariel Rossanigo**

git clone git@github.com:arielrossanigo/machine_learning_interpretability.com

Presentation: https://docs.google.com/presentation/d/18ygxbkkOcny9G7RrrG4tAVPSPEKXo897r2_Am0WdIWQ

### Quien soy?

* Ingeniero en computación
* Profe de Inteligencia Artificial y Machine Learning
* Desarrollador, Data Scientist
* Co-Founder de Bloom AI

### Explainable / Interpretable AI

https://christophm.github.io/interpretable-ml-book/



#### *Interpretability is the degree to which a human can understand the cause of a decision.*

#### *Interpretability is the degree to which a human can consistently predict the model's result* 



### Imports iniciales y lectura del dataset

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import os 
import urllib.request

import pandas as pd
import numpy as np
from zipfile import ZipFile

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'

In [ ]:
if not os.path.exists('Bike-Sharing-Dataset.zip'):
    print("Downloading dataset...")
    urllib.request.urlretrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip",
                               "Bike-Sharing-Dataset.zip")
    print("Dataset downloaded")

In [ ]:
with ZipFile('Bike-Sharing-Dataset.zip') as myzip:
    by_hour = pd.read_csv(myzip.open('hour.csv'))

In [ ]:
train, test = train_test_split(by_hour, test_size=0.3, random_state=42)
train = train.reset_index(drop=True).copy()
test = test.reset_index(drop=True).copy()

features_names = ['season', 'yr', 'hr', 'holiday', 'weekday', 
                  'workingday', 'temp', 'hum', 'windspeed']

In [ ]:
X_train = train[features_names].values
y_train = train.cnt.values

X_test = test[features_names].values
y_test = test.cnt.values

### Modelos interpretables en su naturaleza

#### Regresión lineal

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression().fit(X_train, y_train)
effects = lr.coef_ * X_train

In [ ]:
ax = sns.barplot(y=features_names, x=lr.coef_, orient='h')
ax.set_title('Pesos de la regresión');

In [ ]:
eff = pd.DataFrame(effects, columns=features_names)
eff = eff.unstack().to_frame().reset_index()
eff.columns = ['Feature', '_', 'Value']

ax = sns.boxplot(data=eff, x='Value', y='Feature')
ax.set_title('Effect plot');

#### Arboles de decisión

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *

In [ ]:
the_tree = DecisionTreeRegressor(max_depth=3)
the_tree.fit(X_train, y_train);

In [ ]:
X_sample, _, y_sample, _ = train_test_split(X_test, y_test, train_size=1000, random_state=42)

In [ ]:
case = X_sample[0]

viz = dtreeviz(the_tree, X_sample, y_sample, target_name='# rental bikes', 
               feature_names=features_names, 
               orientation ='TD', fancy=False)
viz.view()

In [ ]:
case = X_sample[0]

viz = dtreeviz(the_tree, X_sample, y_sample, target_name='# rental bikes', 
               feature_names=features_names, 
               orientation ='TD', X=case)
viz.view()

In [ ]:
viz = dtreeviz(the_tree, X_sample, y_sample, target_name='# rental bikes', 
               feature_names=features_names, 
               orientation ='TD', X=case, show_just_path=True)
viz.view()

In [ ]:
print(explain_prediction_path(the_tree, case, feature_names=features_names, explanation_type="plain_english"))

In [ ]:
explain_prediction_path(the_tree, case, 
                        feature_names=features_names, 
                        explanation_type="sklearn_default");

In [ ]:
viz_leaf_target(the_tree, X_sample, y_sample, target_name='# rental bikes', feature_names=features_names, 
                figsize=(15, 8))

### Global Model Agnostic Methods

#### Partial dependence plots


In [ ]:
from sklearn.inspection import plot_partial_dependence
# from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [ ]:
clf = XGBRegressor()
clf.fit(X_train, y_train);

In [ ]:
# from sklearn.metrics import Par

In [ ]:
features = [2, 6, (2, 6)]
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
plot_partial_dependence(clf, X_train, features, ax=ax, feature_names=features_names);

#### Permutation feature importance

* Viene incluido en sklearn (https://scikit-learn.org/stable/modules/permutation_importance.html)

$$i_j = s - \frac{1}{K} \sum_{k=1}^{K} s_{k,j}$$

* Vamos a usar eli5 que ya trae algunas cosas implementadas

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
perm = PermutationImportance(clf, scoring='neg_mean_squared_error', random_state=1, cv="prefit")
perm.fit(X_train, y_train)
eli5.show_weights(perm, feature_names=features_names)

In [ ]:
eli5.show_prediction(clf, X_test[25], show_feature_values=True, feature_names=features_names)

### Local Model Agnostic Methods

####  Local Surrogate (LIME)

*Local interpretable model-agnostic explanations*

The recipe for training local surrogate models:

* Select your instance of interest for which you want to have an explanation of its black box prediction.
* Perturb your dataset and get the black box predictions for these new points.
* Weight the new samples according to their proximity to the instance of interest.
* Train a weighted, interpretable model on the dataset with the variations.
* Explain the prediction by interpreting the local model.

In [ ]:
from lime.lime_tabular import LimeTabularExplainer
lime_explainer = LimeTabularExplainer(X_train, feature_names=features_names, 
                                      class_names=['# rental bikes'], 
                                      verbose=True, mode='regression')

exp = lime_explainer.explain_instance(X_test[42], clf.predict, num_features=9)
exp.show_in_notebook(show_table=True)

#### SHAP (SHapley Additive exPlanations)

In [ ]:
import shap
shap.initjs()

In [ ]:
shap_explainer = shap.TreeExplainer(clf)
shap_values = shap_explainer.shap_values(X_test)

shap.summary_plot(shap_values, feature_names=features_names, plot_type='bar')

In [ ]:
shap.summary_plot(shap_values, features=X_test, feature_names=features_names)

In [ ]:
i = 42
shap.force_plot(shap_explainer.expected_value,
                shap_values[i], 
                features=X_test[i], feature_names=features_names)

In [ ]:
shap.decision_plot(shap_explainer.expected_value, shap_values[i], features=X_test[i], feature_names=features_names)